# Notebook to check integral:

This notebook describes how I get from ERF for CO2 to $\Delta T$ for CO2 using this equation:

\begin{align*}
\Delta T_x (t) &= \int_0^t ERF_x(t') IRF(t-t') dt' \\
\end{align*}

#### The Impulse response function (IRF):
In these calculations we use:
\begin{align*}
IRF(t) = \frac{q_1}{d_1} \exp\Big(\frac{-t}{d_1}\Big) + \frac{q_2}{d_2} \exp\Big(\frac{-t}{d_2}\Big)
\end{align*}

Where the constants, $q_i$ and $d_i$ are shown below. 



In [69]:
import pandas as pd
import xarray as xr
from IPython.display import clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
from ar6_ch6_rcmipfigs.constants import INPUT_DATA_DIR

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Get IRF parameters 

In [70]:
fn_IRF_constants = INPUT_DATA_DIR /'recommended_irf_from_2xCO2_2021_02_25_222758.csv'

irf_consts = pd.read_csv(fn_IRF_constants).set_index('id')

ld1 = 'd1 (yr)'
ld2 = 'd2 (yr)'
lq1 = 'q1 (K / (W / m^2))'
lq2 = 'q2 (K / (W / m^2))'
median = 'median'
perc5 = '5th percentile'
perc95 = '95th percentile'
recommendation = 'recommendation'
irf_consts  # [d1]

,C (W yr / m^2 / K),C_d (W yr / m^2 / K),alpha (W / m^2 / K),eta (dimensionless),kappa (W / m^2 / K),d1 (yr),d2 (yr),q1 (K / (W / m^2)),q2 (K / (W / m^2)),efficacy (dimensionless),ecs (K),tcr (K),rf2xCO2 (W / m^2)
id,,,,,,,,,,,,,
recommendation,7.649789,147.168593,1.31,1.027856,0.880636,3.424102,285.003478,0.443768,0.319591,1.027856,3.0,1.801052,3.93


In [71]:
# lets get the irf values from 0 until i
d1 = float(irf_consts[ld1])
d2 = float(irf_consts[ld2])
q1 = float(irf_consts[lq1])
q2 = float(irf_consts[lq2])

print(f'd1={d1}, d2={d2}, q1={q1}, q2={q2}')

d1=3.4241020923110033, d2=285.0034778419114, q1=0.4437677288834471, q2=0.31959104974250885


### Path input data ERF

In [72]:
from ar6_ch6_rcmipfigs.constants import OUTPUT_DATA_DIR, RESULTS_DIR

#PATH_DATASET = OUTPUT_DATA_DIR / 'ERF_data.nc'
PATH_DATASET = OUTPUT_DATA_DIR/'historic_delta_GSAT/hist_ERF_est.csv'

### Open ERF dataset:

In [73]:
df = pd.read_csv(PATH_DATASET, index_col=0)
da_ERF = df.to_xarray().to_array()#'variable'
da_ERF = da_ERF.rename({'index':'year'})
#ds = xr.open_dataset(PATH_DATASET).sel(year=slice(1700, 2200))  # we need only years until 1700
ds = xr.Dataset({'ERF':da_ERF})
ds
#da_ERF = ds['ERF']

<xarray.Dataset>
Dimensions:   (variable: 10, year: 270)
Coordinates:
  * year      (year) int64 1750 1751 1752 1753 1754 ... 2015 2016 2017 2018 2019
  * variable  (variable) <U3 'CO2' 'N2O' 'CH4' 'NOx' ... 'OC' 'NH3' 'VOC' 'HC'
Data variables:
    ERF       (variable, year) float64 0.0 0.001126 0.002252 ... 0.205 0.2064

In [74]:
# name of output variable
name_deltaT = 'Delta T'

climatemodel = 'climatemodel'
scenario = 'scenario'
variable = 'variable'
time = 'time'
percentile = 'percentile'

In [75]:
ds['ERF'].to_pandas().transpose().to_csv('ERF_timeseries.csv')

### Simple pre-processing

In [76]:
ds['time'] = pd.to_datetime(ds['year'].to_pandas().index.map(str), format='%Y')

# delta_t is 1 (year)
ds['delta_t'] = xr.DataArray(np.ones(len(ds['year'])), dims='year', coords={'year': ds['year']})
ds['year']

<xarray.DataArray 'year' (year: 270)>
array([1750, 1751, 1752, ..., 2017, 2018, 2019])
Coordinates:
  * year     (year) int64 1750 1751 1752 1753 1754 ... 2015 2016 2017 2018 2019

# Integrate from ERF to delta T



## IRF function: 

In [77]:

def IRF(t, d1, q1, d2, q2):
    """
    Returns the IRF function for:
    :param q2:
    :param d2:
    :param q1:
    :param d1:
    :param t: Time in years
    :return:
    IRF
    """
    #print(f'd1={d1}, d2={d2}, q1={q1}, q2={q2}')
    irf = q1 / d1 * np.exp(-t / d1) + q2 / d2 * np.exp(-t / d2)
    return irf
    # l * (alpha1 * np.exp(-t / tau1) + alpha2 * np.exp(-t / tau2))



Calculate for 2019, meaning integrating over i years:

In [78]:
i = 2019-1750

## select only CO2 for check:

In [79]:
ds_in = ds.copy().sel(variable='CO2')

### slice data to contain only years before year i

In [80]:
erf ='ERF'
ds_short = ds_in[{'year': slice(0, i + 1)}].copy()
len(ds_short['ERF'])+1750

2020

In [81]:
# lets get the current year
current_year = ds_short['year'][{'year': i}]  # .dt.year
current_year

<xarray.DataArray 'year' ()>
array(2019)
Coordinates:
    year      int64 2019
    variable  <U3 'CO2'

### IRF is calculated for t-t': calculates t-t'

In [88]:
# lets get a list of years
_years = ds_short['year']  # .dt.year
# lets get the year delta until current year(i)
ds_short['end_year_delta'] = current_year - _years +.5
print(ds_short)

<xarray.Dataset>
Dimensions:         (year: 270)
Coordinates:
  * year            (year) int64 1750 1751 1752 1753 ... 2016 2017 2018 2019
    variable        <U3 'CO2'
Data variables:
    ERF             (year) float64 0.0 0.001126 0.002252 ... 1.982 2.019 2.058
    time            (year) datetime64[ns] 1750-01-01 1751-01-01 ... 2019-01-01
    delta_t         (year) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    end_year_delta  (year) float64 269.5 268.5 267.5 266.5 ... 3.5 2.5 1.5 0.5
    irf             (year) float64 0.0004364 0.0004379 ... 0.09789 0.1307
    to_integrate    (year) float64 0.0 4.931e-07 9.897e-07 ... 0.1977 0.269


### Calculate IRF:

In [89]:
ds_short['irf'] = IRF(
    # t-t':
    ds_short['end_year_delta'] * ds_short['delta_t'], 
    # parameters
    d1, q1, d2, q2)

# The integrand:


\begin{align*}
 ERF_x(t') IRF(t-t') \cdot \Delta t' 
\end{align*}

In [90]:
# lets do the famous integral
ds_short['to_integrate'] = \
        ds_short[erf] * \
        ds_short['irf'] * \
        ds_short['delta_t']

In [91]:
ds_short

<xarray.Dataset>
Dimensions:         (year: 270)
Coordinates:
  * year            (year) int64 1750 1751 1752 1753 ... 2016 2017 2018 2019
    variable        <U3 'CO2'
Data variables:
    ERF             (year) float64 0.0 0.001126 0.002252 ... 1.982 2.019 2.058
    time            (year) datetime64[ns] 1750-01-01 1751-01-01 ... 2019-01-01
    delta_t         (year) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    end_year_delta  (year) float64 269.5 268.5 267.5 266.5 ... 3.5 2.5 1.5 0.5
    irf             (year) float64 0.0004356 0.0004371 ... 0.08474 0.1131
    to_integrate    (year) float64 0.0 4.923e-07 9.88e-07 ... 0.1711 0.2327

### Finally do the sum:

In [92]:
ds_short['to_integrate'].sum()

<xarray.DataArray 'to_integrate' ()>
array(0.96697278)
Coordinates:
    variable  <U3 'CO2'

In [87]:
q2

0.31959104974250885

In [ ]:
(a+da)(db)*dt = 

In [ ]:
(a+da)(b+db)-a*b != 

$f(x) = x^2$

$\int_0^2 f dx$

1*2=2

$\frac{1}{3}x^3= \frac{1}{3}8=2.66$ 

In [2]:
8/3

2.6666666666666665